# Install libraries 


In [6]:
import sys, os, json
import mne, sklearn, wandb
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
from nilearn import datasets, image, masking, plotting
from nilearn.input_data import NiftiLabelsMasker


# animation part
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
# from celluloid import Camera   # it is convinient method to animate
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation


## torch libraries 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset

from pytorch_model_summary import summary


In [3]:
%load_ext autoreload
%autoreload 2
sys.path.insert(1, os.path.realpath(os.path.pardir))

from utils import get_datasets
from utils import preproc
from utils import torch_dataset
from utils import train_utils
from utils import inference
from utils.models_arch import autoencoder_v3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set all hyperparameters
- Cuda and GPU.
- Parameters of dataset. 
- random seed( if necessary). 


In [ ]:
print(torch.cuda.is_available(), torch.cuda.device_count())
torch.cuda.set_device(0)

In [4]:
config = dict(  
                dataset_name = 'NODDI', # NODDI
                fps = 250,
                new_fps=250, 
                n_channels = 30, # 64 
                n_roi = 21,
                test_sec = 60, # in seconds.
                freqs = np.linspace(2, 100, 16), 
                
                
                bold_delay = 0,
                to_many = True,
                random_subsample = True,
                sample_per_epoch = 25600, 
                WINDOW_SIZE = 2048,
                    
                optimizer='adam',
                lr=0.001,
                weight_decay=0.00001, 
                batch_size=256, 

                loss_function = 'mse', 
                model_type = '1D_CNN_AE_changed'
                )


hp_autoencoder = dict(n_electrodes=config['n_channels'],
                      n_freqs = len(config['freqs']),
                      n_channels_out = config['n_roi'],
                      n_res_block=0, 
                      channels=[32, 32, 64, 64], 
                      kernel_sizes=[15, 11, 5, 5],
                      strides=[8, 8, 4]
                     )


config = {**hp_autoencoder, **config}

params_train = {'batch_size': config['batch_size'],
                'shuffle': True,
                'num_workers': 0}

params_val = {'batch_size': config['batch_size'],
              'shuffle': False}

In [ ]:
# import random

# torch.manual_seed(0)
# random.seed(0)  # python operation seed
# np.random.seed(0)

# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True


# Upload preprocessed dataset from np files. 
It should accelerate speed of experiments.

In [7]:
with open("labels_roi.json", 'r') as f:
    labels_roi = json.load(f)


if config['dataset_name']=='CWL':
    dataset_path = '../data/dataset_cwl_250_hz.npz'
    
elif config['dataset_name']=='NODDI':
    dataset_path = '../data/dataset_NODDI_250_hz.npz'
else:
    print('no such dataset')


# download data
data = np.load(dataset_path)
train_dataset_prep = (data['x_train'], data['y_train'])
test_dataset_prep = (data['x_test'], data['y_test'])





# apply time dealy corrected
train_dataset_prep = preproc.bold_time_delay_align(train_dataset_prep, 
                                                   config['new_fps'],
                                                   config['bold_delay'])
test_dataset_prep = preproc.bold_time_delay_align(test_dataset_prep, 
                                                  config['new_fps'],
                                                  config['bold_delay'])


print('Size of train dataset:', train_dataset_prep[0].shape, train_dataset_prep[1].shape)
print('Size of test dataset:', test_dataset_prep[0].shape, test_dataset_prep[1].shape)

# torch dataset creation 
torch_dataset_train = torch_dataset.CreateDataset_eeg_fmri(train_dataset_prep, 
                                                            random_sample=config['random_subsample'], 
                                                            sample_per_epoch=config['sample_per_epoch'], 
                                                            to_many=config['to_many'])

torch_dataset_test = torch_dataset.CreateDataset_eeg_fmri(test_dataset_prep, 
                                                            random_sample=False, 
                                                            sample_per_epoch=None, 
                                                            to_many=config['to_many'])


# init dataloaders for training
train_loader = torch.utils.data.DataLoader(torch_dataset_train, **params_train)
val_loader = torch.utils.data.DataLoader(torch_dataset_test, **params_val)

# Init Model, Loss, optimizers

In [8]:
model = autoencoder_v3.AutoEncoder1D(**hp_autoencoder)

loss_func = train_utils.make_mse_loss()
train_step = train_utils.train_step

optimizer = optim.Adam(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])


print(summary(model, torch.zeros(4, config['n_channels'], 
                                 len(config['freqs']),
                                 config['WINDOW_SIZE']), show_input=False))


-------------------------------------------------------------------------
      Layer (type)          Output Shape         Param #     Tr. Param #
          Conv2d-1     [4, 16, 16, 1024]             496             496
          Conv1d-2         [4, 32, 1024]           8,224           8,224
         Block1D-3          [4, 32, 128]           5,536           5,536
         Block1D-4           [4, 64, 16]          19,264          19,264
         Block1D-5            [4, 64, 4]          21,312          21,312
         Block1D-6            [4, 64, 4]          21,312          21,312
          Conv1d-7            [4, 21, 4]           1,365           1,365
        Upsample-8         [4, 21, 1024]               0               0
Total params: 77,509
Trainable params: 77,509
Non-trainable params: 0
-------------------------------------------------------------------------


/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


# Model training

In [ ]:
n_runs = 3

for i in range(n_runs):
    
    model = autoencoder_v3.AutoEncoder1D(**hp_autoencoder)

    loss_func = train_utils.make_mse_loss()
    train_step = train_utils.train_step

    optimizer = optim.Adam(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])
    
    
    parameters = {
        'EPOCHS': 250,
        'model': model, 
        'train_loader': train_loader, 
        'val_loader': val_loader, 
        'loss_function': loss_func,
        'train_step': train_step,
        'optimizer': optimizer, 
        'device': 'cuda', 
        'raw_test_data': test_dataset_prep,
        'show_info': 1, 
        'num_losses': 5,
        'labels': labels_roi,
        'inference_function': inference.model_inference_function, 
        'to_many': config['to_many']
    }



    path_to_save_wandb = 'common/koval_alvi/Checkpoints/wandb_brain'
    
    
    with wandb.init(project="eeg_fmri", config=config, save_code=True):
        
        wandb.define_metric("val/corr_mean", summary="max")

        if i == 0: 
            exp_name = wandb.run.name
        
        wandb.run.name = exp_name +'_run_' + str(i)
        
        print(config)
        print(parameters['model'])
        print(summary(model, torch.zeros(4, config['n_channels'],
                                         len(config['freqs']), config['WINDOW_SIZE']), show_input=False))
        
        model = train_utils.wanb_train_regression(**parameters)
        

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koval_alvi (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'n_electrodes': 30, 'n_freqs': 16, 'n_channels_out': 21, 'n_res_block': 0, 'channels': [32, 32, 64, 64], 'kernel_sizes': [15, 11, 5, 5], 'strides': [8, 8, 4], 'dataset_name': 'CWL', 'fps': 250, 'new_fps': 250, 'n_channels': 30, 'n_roi': 21, 'test_sec': 60, 'freqs': array([  2.        ,   8.53333333,  15.06666667,  21.6       ,
        28.13333333,  34.66666667,  41.2       ,  47.73333333,
        54.26666667,  60.8       ,  67.33333333,  73.86666667,
        80.4       ,  86.93333333,  93.46666667, 100.        ]), 'bold_delay': 5, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 25600, 'WINDOW_SIZE': 1024, 'optimizer': 'adam', 'lr': 0.001, 'weight_decay': 1e-05, 'batch_size': 512, 'loss_function': 'mse', 'model_type': '1D_CNN_AE_changed'}
AutoEncoder1D(
  (spatial_reduce_2d): Conv2d(30, 16, kernel_size=(1, 1), stride=(1, 1))
  (spatial_reduce): Conv1d(256, 32, kernel_size=(1,), stride=(1,))
  (downsample_blocks): ModuleList(
    (0): Block1D(
      (downsample): AvgPool1

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


..................................................
Epoch 1 train loss_0 : 0.905 val loss_0 : 1.29 train loss_1 : 0.25 val loss_1 : 0.0794 train loss_2 : 0.0 val loss_2 : 0.0 train loss_3 : 0.0 val loss_3 : 0.0 train loss_4 : 0.0 val loss_4 : 0.0 
.

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


...

.................................................
Epoch 59 train loss_0 : 0.605 val loss_0 : 1.38 train loss_1 : 0.645 val loss_1 : 0.0748 
.

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


.................................................
Epoch 60 train loss_0 : 0.607 val loss_0 : 1.38 train loss_1 : 0.645 val loss_1 : 0.0746 
.

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


.................................................

# 